In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
file_dir = "C://Users/riley/documents/Data Analytics/Movies-ETL"
with open (f'{file_dir}/wikipedia-movies.json', mode= 'r') as file:
    wikiMoviesRaw = json.load(file)

In [3]:
kaggleMetadata = pd.read_csv(f'{file_dir}/archive/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}/archive/ratings.csv')
wikiMoviesdf = pd.DataFrame(wikiMoviesRaw)

In [4]:
wikiMovies = [movie for movie in wikiMoviesRaw
             if("Director" in movie or "Directed by" in movie) and "imdb_link" in movie
             and "No. of episodes" not in movie]
len(wikiMovies)

7076

In [5]:
def cleanMovie(movie):
    movie = dict(movie)
    altTitles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            altTitles[key] = movie[key]
            movie.pop(key)
    if len(altTitles) > 0:
        movie['alt_titles'] = altTitles
    def changeColumnName(oldName, newName):
        if oldName in movie:
            movie[newName] = movie.pop(oldName)
        changeColumnName('Adaptation by', 'Writer(s)')
    changeColumnName('Country of origin', 'Country')
    changeColumnName('Directed by', 'Director')
    changeColumnName('Distributed by', 'Distributor')
    changeColumnName('Edited by', 'Editor(s)')
    changeColumnName('Length', 'Running time')
    changeColumnName('Original release', 'Release date')
    changeColumnName('Music by', 'Composer(s)')
    changeColumnName('Produced by', 'Producer(s)')
    changeColumnName('Producer', 'Producer(s)')
    changeColumnName('Productioncompanies ', 'Production company(s)')
    changeColumnName('Productioncompany ', 'Production company(s)')
    changeColumnName('Released', 'Release Date')
    changeColumnName('Release Date', 'Release date')
    changeColumnName('Screen story by', 'Writer(s)')
    changeColumnName('Screenplay by', 'Writer(s)')
    changeColumnName('Story by', 'Writer(s)')
    changeColumnName('Theme music composer', 'Composer(s)')
    changeColumnName('Written by', 'Writer(s)')
    
    return movie

In [ ]:
cleanMovies = [cleanMovie(movie) for movie in wikiMovies]
wikiMoviesdf = pd.DataFrame(cleanMovies)
sorted(wikiMoviesdf.columns.tolist())

In [ ]:
wikiMoviesdf['imdb_id'] = wikiMoviesdf['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wikiMoviesdf))
wikiMoviesdf.drop_duplicates(subset = 'imdb_id', inplace = True)
print(len(wikiMoviesdf))
wikiMoviesdf.head()

In [ ]:
wikiColumnstoKeep = [column for column in wikiMoviesdf.columns if wikiMoviesdf[column].isnull().sum() < len(wikiMoviesdf) * 0.9]
wikiMoviesdf = wikiMoviesdf[wikiColumnstoKeep]

In [ ]:
boxOffice = wikiMoviesdf['Box office'].dropna()

In [ ]:
boxOffice[boxOffice.map(lambda x: type(x) != str)]

In [ ]:
boxOffice = boxOffice.apply(lambda x: ' '.join(x) if type(x) == list else x)
import re

In [ ]:
formOne = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
formTwo = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
matchesFormOne = boxOffice.str.contains(formOne, flags=re.IGNORECASE, na = False).sum()

In [ ]:
matchesFormTwo = boxOffice.str.contains(formTwo, flags=re.IGNORECASE, na=False).sum()

In [ ]:
boxOffice[~matchesFormOne & ~matchesFormTwo]

In [1]:
boxOffice = boxOffice.str.replace(r'\$.*[---](?![a-z])', '$', regex = True)

NameError: name 'boxOffice' is not defined

In [2]:
boxOffice.str.extract(f'({formOne}|{formTwo})')

NameError: name 'boxOffice' is not defined

In [3]:
def parseDollars(s):
    if type(s) != str:
        return np.nan

    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        value = float(s) * 10**6

        return value

    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        value = float(s) * 10**9

        return value

    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        s = re.sub('\$|,','', s)
        value = float(s)

        return value

    else:
        return np.nan

In [4]:
wikiMoviesdf['box_office'] = boxOffice.str.extract(f'({formOne}|{formTwo})', flags=re.IGNORECASE)[0].apply(parseDollars)
wikiMoviesdf.drop('Box office', axis = 1, inplace = True)

NameError: name 'boxOffice' is not defined

In [5]:
budget = wikiMoviesdf['Budget'].dropna()
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

NameError: name 'wikiMoviesdf' is not defined

In [6]:
matchesFormOne = budget.str.contains(formOne, flags=re.IGNORECASE, na=False)
matchesFormTwo = budget.str.contains(formTwo, flags=re.IGNORECASE, na=False)
budget[~matchesFormOne & ~matchesFormTwo]

NameError: name 'budget' is not defined

In [7]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matchesFormOne & ~matchesFormTwo]

NameError: name 'budget' is not defined

In [8]:
wikiMoviesdf['budget'] = budget.str.extract(f'({formOne}|{formTwo})', flags=re.IGNORECASE)[0].apply(parseDollars)
wikiMoviesdf.drop('Budget', axis = 1, inplace = True)

NameError: name 'budget' is not defined

In [9]:
releaseDate = wikiMoviesdf['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

NameError: name 'wikiMoviesdf' is not defined

In [10]:
dateFormOne = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
dateFormTwo = r'\d{4}.[01]\d.[0123]\d'
dateFormThree = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
dateFormFour = r'\d{4}'

In [11]:
releaseData.str.extract(f'({dateFormOne}|{dateFormTwo}|{dateFormThree}|{dateFormFour})', flags=re.IGNORECASE)

NameError: name 'releaseData' is not defined

In [12]:
wikiMoviesdf['release_date'] = pd.to_datetime(releaseDate.str.extract(f'({dateFormOne}|{dateFormTwo}|{dateFormThree}|{dateFormFour})')[0], infer_datetime_format=True)

NameError: name 'pd' is not defined

In [13]:
runningTime = wikiMoviesdf['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
runningTime.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

NameError: name 'wikiMoviesdf' is not defined

In [14]:
runningTime[runningTime.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

NameError: name 'runningTime' is not defined

In [15]:
runningTime.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()
runningTime[runningTime.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

NameError: name 'runningTime' is not defined

In [17]:
runningTimeExtract = runningTime.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
runningTimeExtract = runningTimeExtract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

NameError: name 'runningTime' is not defined

In [18]:
wikiMoviesdf['running_time'] = runningTimeExtract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
wikiMoviesdf.drop('Running time', axis=1, inplace = True)

NameError: name 'runningTimeExtract' is not defined

In [19]:
kaggleMetadata['adult'].value_counts()
kaggleMetadata[~kaggleMetadata['adult'].isin(['True','False'])]

NameError: name 'kaggleMetadata' is not defined

In [20]:
kaggleMetadata = kaggleMetadata[kaggleMetadata['adult'] == 'False'].drop('adult',axis='columns')

NameError: name 'kaggleMetadata' is not defined

In [22]:
kaggleMetadata['video'] == 'True'

NameError: name 'kaggleMetadata' is not defined

In [23]:
kaggleMetadata['video'] = kaggleMetadata['video'] == 'True'
kaggleMetadata['budget'] = kaggleMetadata['budget'].astype(int)
kaggleMetadata['id'] = pd.to_numeric(kaggleMetadata['id'], errors='raise')
kaggleMetadata['popularity'] = pd.to_numeric(kaggleMetadata['popularity'], errors='raise')
kaggleMetadata['release_date'] = pd.to_datetime(kaggleMetadata['release_date'])

NameError: name 'kaggleMetadata' is not defined

In [24]:
ratings.info(null_counts=True)
pd.to_datetime(ratings['timestamp'], unit='s')
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

NameError: name 'ratings' is not defined

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()